In [2]:
import pandas as pd
import os

# 1. Dados

In [3]:
path = 'data/raw'

dados_antigos = pd.read_csv(os.path.join(path, 'pgm-dataset.csv'), dtype=str)
dados_novos_janeiro_maio = pd.read_csv(os.path.join(path, 'novo_dataframe_com_metadados.csv'), dtype=str)

# Tratando os dados novos
dados_novos_janeiro_maio['data_aviso'] = dados_novos_janeiro_maio['data_aviso'].apply(lambda x: x.split('.')[0])

# Tratando os dados antigos
dados_antigos_filtered = dados_antigos[['idAviso', 'dataAviso', 'teorTexto', 'assuntos', 'classeProcesso', 'orgaoJulgador', 'setorDestino']].copy()
dados_antigos_filtered.rename(columns={
    'idAviso':'id_aviso',
    'dataAviso':'data_aviso',
    'teorTexto': 'teor_texto',
    'classeProcesso':'cod_classe_processual',
    'orgaoJulgador':'orgao_julgador',
    'setorDestino':'setor_destino'
}, inplace=True)

## 1.1 - Merge dos DataFrames

In [10]:
print(f'Shape dados antigos: {dados_antigos_filtered.shape}')
print(f'Shape dados novos: {dados_novos_janeiro_maio.shape}')

df_merged = pd.concat([dados_antigos_filtered, dados_novos_janeiro_maio])

# Filtrando apenas as linhas com setor_destino válido
df_merged = df_merged[df_merged['setor_destino'].notna()].copy()

df_merged.sort_values(by='data_aviso')

Shape dados antigos: (15942, 7)
Shape dados novos: (34903, 8)


,id_aviso,data_aviso,teor_texto,assuntos,cod_classe_processual,orgao_julgador,setor_destino,id_processo
12745,2482609,2024-04-01 14:02:27,Poder Judiciário do Rio Grande do NorteTerceir...,5951,198,Gab. da Vice-Presidência no Pleno,Procuradoria Fiscal,NaN
12744,2482461,2024-04-01 14:02:35,Poder Judiciário do Rio Grande do NorteTerceir...,10536;5952,1116,2ª Vara de Execução Fiscal e Tributária de Natal,Procuradoria Fiscal,NaN
12743,2482428,2024-04-01 14:03:44,Poder Judiciário do Rio Grande do NorteTerceir...,5946,198,Gab. Des. Vivaldo Pinheiro na Câmara Cível,Procuradoria Fiscal,NaN
12752,2484815,2024-04-01 23:19:19,Poder Judiciário do Rio Grande do NortePrimeir...,5952,7,6ª Vara de Execução Fiscal e Tributária de Natal,Procuradoria Fiscal,NaN
12750,2484537,2024-04-01 23:19:23,Poder Judiciário do Rio Grande do NortePrimeir...,9178,1118,4ª Vara de Execução Fiscal e Tributária de Natal,CHEFIA FISCAL,NaN
...,...,...,...,...,...,...,...,...
14862,23210879,2025-05-19 03:00:19,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,9517;10706,12078,1º Juizado da Fazenda Pública da Comarca de Natal,DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE,08132991120238205001
8039,23211840,2025-05-19 03:00:19,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10536;10395,7,5ª Vara de Execução Fiscal e Tributária de Natal,Chefia Meio Ambiente,08739536120238205001
5176,23213095,2025-05-19 03:00:19,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,5951,1116,3ª Vara de Execução Fiscal e Tributária de Natal,APOIO FISCAL,08032331620168205001
2053,23217735,2025-05-19 03:00:19,PODER\nJUDICIÁRIO DO ESTADO DO RIO GRANDE DO N...,5988,1118,6ª Vara de Execução Fiscal e Tributária de Natal,Procuradoria Fiscal,08809477120248205001


## 1.2 - Filtrando os dados baseado no setor_destino

In [33]:
# Corrigindo erros
df_merged['setor_destino'] = df_merged['setor_destino'].apply(lambda x: x.upper())
df_merged.replace(
    {
        'DEPARTAMENTO DE C�LCULOS E CONTABILIDADE': 'DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE',
        'PROCURADORIA DA SA�DE': 'PROCURADORIA DA SAÚDE',
    },
    inplace=True
)

In [ ]:
# Filtrando classes
classes = [
    'PROCURADORIA JUDICIAL',
    'CHEFIA JUDICIAL',
    'PROCURADORIA ADMINISTRATIVA',
    'CHEFIA ADMINISTRATIVA',
    'PROCURADORIA FISCAL',
    'APOIO FISCAL',
    'PROCURADORIA DO MEIO AMBIENTE',
    'CHEFIA MEIO AMBIENTE',
    'PROCURADORIA PATRIMONIAL',
    'CHEFIA PATRIMONIAL',
    'PROCURADORIA DA SAÚDE',
    'DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE',
    'REQUISITÓRIO DE PAGAMENTO - CONTABILIDADE'
]

df_filtrado = df_merged[df_merged['setor_destino'].isin(classes)].copy()

print(f'Essa filtragem corresponde a cerca de {len(df_filtrado)/len(df_merged)*100:.2f}% do dataset original')

Essa filtragem corresponde a cerca de 99.53% do dataset original


Juntando classes como:

* Chefia Adminsitrativa + Procuradoria Administrativa
* Chefia Judicial + Procuradoria Judicial
* Chefia Meio Ambiente + Procuradoria do Meio Ambiente
* Chefia Patrimonial + Procuradoria Patrimonial
* Departamento de Cálculos e Contabilidade + Requisitório de Pagamentos - Contabilidade

In [39]:
swap_dict = {
    'CHEFIA ADMINISTRATIVA': 'PROCURADORIA ADMINISTRATIVA',
    'CHEFIA JUDICIAL': 'PROCURADORIA JUDICIAL',
    'CHEFIA MEIO AMBIENTE': 'PROCURADORIA DO MEIO AMBIENTE',
    'CHEFIA PATRIMONIAL': 'PROCURADORIA PATRIMONIAL',
    'REQUISITÓRIO DE PAGAMENTO - CONTABILIDADE': 'DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE'
}

# Substituindo valores acima
df_filtrado.replace(swap_dict, inplace=True)

df_filtrado['setor_destino'].value_counts(normalize=True)

setor_destino
PROCURADORIA ADMINISTRATIVA                 0.353437
APOIO FISCAL                                0.287788
PROCURADORIA FISCAL                         0.134338
PROCURADORIA JUDICIAL                       0.094297
DEPARTAMENTO DE CÁLCULOS E CONTABILIDADE    0.079623
PROCURADORIA DA SAÚDE                       0.036963
PROCURADORIA DO MEIO AMBIENTE               0.007597
PROCURADORIA PATRIMONIAL                    0.005957
Name: proportion, dtype: float64

# 2. Separação dos dados

In [44]:
from sklearn.model_selection import train_test_split

X, y = df_filtrado[['teor_texto', 'assuntos', 'cod_classe_processual', 'orgao_julgador']], df_filtrado['setor_destino']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

print(f'Train shape: {X_train.shape}')
print(f'Test shape: {X_test.shape}')


Train shape: (40018, 4)
Test shape: (10005, 4)


In [47]:
X_train.head()

,teor_texto,assuntos,cod_classe_processual,orgao_julgador
23217,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10536,1116,6ª Vara de Execução Fiscal e Tributária de Natal
4130,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299;10305,156,5ª Vara da Fazenda Pública da Comarca de Natal
29432,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299,12078,3ª Vara da Fazenda Pública da Comarca de Natal
27670,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,5951,1116,4ª Vara de Execução Fiscal e Tributária de Natal
5986,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,10299,12078,1º Juizado da Fazenda Pública da Comarca de Natal


# 3. Pré-processamentos

### 3.1 - Orgão Julgador

In [99]:
import re

def preprocessa_orgao_julgador(text: str) -> str:
    
    if not text or not isinstance(text, str): # Checa se é None
        return text
    
    patterns_to_remove = [
        r'\s+da Comarca.*',
        #r'\s+de Natal',
        r'\d{1,2}[ºª]\s',
    ]

    for pattern in patterns_to_remove:
        text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    return text

X_train['orgao_julgador'] = X_train['orgao_julgador'].apply(preprocessa_orgao_julgador)
X_test['orgao_julgador'] = X_test['orgao_julgador'].apply(preprocessa_orgao_julgador)

### 3.2 - Teor Texto

In [100]:
def preprocessa_teor_texto(text: str) -> str:
    """
    Remove cabeçalho (palavras-chave do topo) e rodapé (Natal/RN com variações)
    do 'text' e retorna o resultado em minúsculas.
    - A detecção do cabeçalho é *sensível a maiúsculas/minúsculas* (como no seu código).
    - A detecção do rodapé é *case-insensitive* e tolera espaços entre letras.
    """

    if not isinstance(text, str):
        return text
    
    # Palavras-chave do cabeçalho
    palavras_cabecalho = [
        'ATO ORDINATÓRIO', 'DESPACHO', 'CERTIDÃO', 'DECISÃO', 'SENTENÇA', 'INTIMAÇÃO'
    ]

    # Constrói um padrão "folgado" que aceita espaços entre TODAS as letras
    # Ex.: "ATO ORDINATÓRIO" -> A\s*T\s*O\s*O\s*R\s*D...

    def padrao_flex(termo: str) -> str:
        s = ''.join(termo.split())
        return r'(?<!\w)' + r'\s*'.join(map(re.escape, s)) + r'(?!\w)'
    
    # Cabeçalho: Alternância de todos os termos
    re_cabecalho = re.compile('|'.join(padrao_flex(t) for t in palavras_cabecalho))
    m = re_cabecalho.search(text)

    if m:
        text = text[m.end():] # corta tudo até o fim do match do cabeçalho

    # Rodapé: "Natal/RN" com ou sem espaços e case-insensitive
    re_rodape = re.compile(r'N\s*A\s*T\s*A\s*L\s*/\s*R\s*N', flags=re.IGNORECASE)
    r = re_rodape.search(text)
    if r:
        text = text[:r.start()] # corta a partir do inicio do rodapé

    text = re.sub(r'\s+', ' ', text)

    return text.strip().lower()

X_train['teor_texto'] = X_train['teor_texto'].apply(preprocessa_teor_texto)
X_test['teor_texto'] = X_test['teor_texto'].apply(preprocessa_teor_texto)